# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *
from pyspark.ml.feature import SQLTransformer

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'sqlOptionsAss'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [7]:
# load the data
fil = '../../data/googleplaystore.csv'
schem = StructType([StructField('App', StringType()), StructField('Category', StringType()),
                    StructField('Rating', FloatType()), StructField('Reviews', IntegerType()),
                    StructField('Size', StringType()), StructField('Installs', StringType()),
                    StructField('Type', StringType()), StructField('Price', FloatType()),
                    StructField('Content Rating', StringType()), StructField('Genres', StringType()),
                    StructField('Last Updated', DateType()), StructField('Current Ver', StringType()),
                    StructField('Android Ver', StringType())])

goog = spark.read.format('csv').options(header=True, dateFormat='MMMM d, yyyy').schema(schem).load(fil)\
    .withColumnRenamed('Content Rating', 'ContentRat').withColumnRenamed('Last Updated', 'Updated')\
    .withColumnRenamed('Current Ver', 'Version').withColumnRenamed('Android Ver', 'AndroidVersion').cache()

# talk
print('%d records'%goog.count())
display(goog.limit(10).toPandas())

10841 records


,App,Category,Rating,Reviews,Size,Installs,Type,Price,ContentRat,Genres,Updated,Version,AndroidVersion
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0.0,Everyone,Art & Design,2017-03-26,1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0.0,Everyone,Art & Design,2018-04-26,1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0.0,Everyone,Art & Design,2018-06-14,6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0.0,Everyone,Art & Design,2017-09-20,2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0.0,Everyone,Art & Design;Creativity,2018-07-03,2.8,4.0.3 and up


## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [ ]:
# already did this

As well as the schema to make sure all the column types were correctly infered

In [ ]:
# already set the schema

Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [ ]:
# don't need this
'''from pyspark.sql.types import IntegerType, FloatType
newdf = df.withColumn("Rating", df["Rating"].cast(FloatType())) \
            .withColumn("Reviews", df["Reviews"].cast(IntegerType())) \
            .withColumn("Price", df["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()'''

Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [8]:
goog.createOrReplaceTempView('tempGoog')

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [11]:
spark.sql('select * from tempGoog where Rating > 4.1;').show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+----------+--------------------+----------+------------------+--------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|ContentRat|              Genres|   Updated|           Version|AndroidVersion|
+--------------------+--------------+------+-------+----+-----------+----+-----+----------+--------------------+----------+------------------+--------------+
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|  0.0|  Everyone|        Art & Design|2018-08-01|             1.2.4|  4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|  0.0|      Teen|        Art & Design|2018-06-08|Varies with device|    4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|Free|  0.0|  Everyone|Art & Design;Crea...|2018-06-20|               1.1|    4.4 and up|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|

## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [23]:
res = spark.sql("select App, Rating from tempGoog where (Category = 'COMICS') and (Rating > 4.5);")
res.show(5, truncate=False)

+--------------------------------------------------------------------+------+
|App                                                                 |Rating|
+--------------------------------------------------------------------+------+
|Manga Master - Best manga & comic reader                            |4.6   |
|GANMA! - All original stories free of charge for all original comics|4.7   |
|Röhrich Werner Soundboard                                           |4.7   |
|Unicorn Pokez - Color By Number                                     |4.8   |
|Manga - read Thai translation                                       |4.6   |
+--------------------------------------------------------------------+------+
only showing top 5 rows



## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [18]:
spark.sql('select Category, sum(Reviews) as totReview from tempGoog group by Category order by sum(Reviews) desc;').show(10)

+---------------+----------+
|       Category| totReview|
+---------------+----------+
|           GAME|1585422349|
|  COMMUNICATION| 815462260|
|         SOCIAL| 621241422|
|         FAMILY| 410226330|
|          TOOLS| 273185044|
|    PHOTOGRAPHY| 213516650|
|       SHOPPING| 115041222|
|   PRODUCTIVITY| 114116975|
|  VIDEO_PLAYERS| 110380188|
|PERSONALIZATION|  89346140|
+---------------+----------+
only showing top 10 rows



## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [24]:
spark.sql('select App, Reviews from tempGoog order by Reviews desc limit 1;').show()

+--------+--------+
|     App| Reviews|
+--------+--------+
|Facebook|78158306|
+--------+--------+



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [25]:
display(spark.sql('select * from tempGoog where App like "%dating%" order by Reviews desc;').limit(10).toPandas())

,App,Category,Rating,Reviews,Size,Installs,Type,Price,ContentRat,Genres,Updated,Version,AndroidVersion
0,happn – Local dating app,LIFESTYLE,4.3,1118201,Varies with device,"10,000,000+",Free,0.0,Mature 17+,Lifestyle,2018-07-24,Varies with device,Varies with device
1,Princess Closet : Otome games free dating sim,FAMILY,4.5,29495,56M,"1,000,000+",Free,0.0,Teen,Simulation,2018-05-24,1.11.0,4.0.3 and up
2,"Meet, chat & date. Free dating app - Chocolate...",DATING,3.9,8661,9.5M,"1,000,000+",Free,0.0,Mature 17+,Dating,2018-04-03,0.1.11,4.0 and up
3,Friend Find: free chat + flirt dating app,DATING,NaN,23,11M,100+,Free,0.0,Mature 17+,Dating,2018-07-31,1.0,4.4 and up
4,Spine- The dating app,DATING,5.0,5,9.3M,500+,Free,0.0,Teen,Dating,2018-07-14,4.0,4.0.3 and up


## 6. Use SQL Transformer to display how many free apps there are in this list

In [28]:
sqlTrans = SQLTransformer(statement='select count(*) from tempGoog where Type="Free";')
sqlTrans.transform(goog).show()

+--------+
|count(1)|
+--------+
|   10037|
+--------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [29]:
res = spark.sql('select Genres, count(*) as Cnt from tempGoog group by Genres order by count(*) desc;')
res.show(1)

+------+---+
|Genres|Cnt|
+------+---+
| Tools|842|
+------+---+
only showing top 1 row



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [32]:
res = goog.select('App', 'Installs', 'Type', 'Price', 'Reviews').\
    where((col('Genres')=='Tools') & (col('Reviews') > 100)).\
    orderBy(col('Reviews').desc())
display(res.toPandas())

,App,Installs,Type,Price,Reviews
0,Clean Master- Space Cleaner & Antivirus,"500,000,000+",Free,0.0,42916526
1,"Security Master - Antivirus, VPN, AppLock, Boo...","500,000,000+",Free,0.0,24900999
2,"360 Security - Free Antivirus, Booster, Cleaner","100,000,000+",Free,0.0,16771865
3,DU Battery Saver - Battery Charger & Battery Life,"100,000,000+",Free,0.0,13479633
4,Cache Cleaner-DU Speed Booster (booster & clea...,"100,000,000+",Free,0.0,12759815
...,...,...,...,...,...
530,DNS Changer - BEST (Gprs/Edge/3G/H/H+/4G),"5,000+",Free,0.0,123
531,CF cal,"10,000+",Free,0.0,123
532,Evolve CP Calc. for PokemonGo,"10,000+",Free,0.0,116
533,JavaScript Editor CR,"5,000+",Free,0.0,114


## That's all folks! Great job!

In [33]:
sc.stop()